In [ ]:
#Import the required packages
import scipy
#Import package pandas for data analysis
import pandas as pd
import pandas.io.sql as psql

#Import package numpy for numeric computing
import numpy as np

#Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt

#Import package seaborn for visualisation
import seaborn as sns

#For showing plots directly in the notebook run the command below
%matplotlib inline

from matplotlib.backends.backend_pdf import PdfPages


import sqlalchemy
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import pickle
import requests
import json
import datetime

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#read feather file
df = pd.read_feather("rt_leavetimes_DB_2018(4).feather")

In [ ]:
df

In [ ]:
df.shape 

df[df['StopPointID']=='261']

In [ ]:
df.nunique()

In [ ]:
#delete not needed columns

del df['NOTE']
del df['LASTUPDATE']
del df['JUSTIFICATIONID']
del df['SUPPRESSED']
del df['PASSENGERS']
del df['PASSENGERSIN']
del df['PASSENGERSOUT']
del df['DISTANCE']
del df['DATASOURCE']


In [ ]:
#change datatypes

df['DAYOFSERVICE'] = df['DAYOFSERVICE'].astype('datetime64[ns]')
df['TRIPID'] = df['TRIPID'].astype('category')
df['PROGRNUMBER'] = df['PROGRNUMBER'].astype('category')
df['STOPPOINTID'] = df['STOPPOINTID'].astype('category')
df['VEHICLEID'] = df['VEHICLEID'].astype('category')

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
df['DayOfService'] = df['DAYOFSERVICE'].astype('datetime64[ns]')

In [ ]:
object_columns = df.select_dtypes(['object']).columns

In [ ]:
for column in object_columns:
    df[column] = df[column].astype('category')

In [ ]:
df.dtypes

In [ ]:
df['ProgrNumber'] = df['PROGRNUMBER'].astype('category')

In [ ]:
df.dtypes

In [ ]:
df.nunique()

In [ ]:
df.isna().sum()

In [ ]:
#read in cleaned Trips Data

trips = pd.read_csv('MasterCleanedTripsData.csv',index_col=False)

In [ ]:
trips

In [ ]:
trips.dtypes

In [ ]:
df.dtypes

In [ ]:
#change datatypes
df['TRIPID'] = df['TRIPID'].astype('int')

In [ ]:
df['TripID'] = df['TRIPID']

In [ ]:
df['DayOfService'] = df['DAYOFSERVICE']

In [ ]:
df['DayOfService'] = df['DayOfService'].astype('datetime64[ns]')

In [ ]:
trips['DayOfService'] = trips['DayOfService'].astype('datetime64[ns]')

In [ ]:
#merge leavetimes and tripdata
merged_df = pd.merge(df,
                 trips[['TripID', 'DayOfService', 'LineID', 'RouteID', 'Direction']],
                 on=['TripID', 'DayOfService'])


In [ ]:
merged_df

In [ ]:
#create new features

merged_df['ArrDel'] = merged_df['PLANNEDTIME_DEP'] - merged_df['ACTUALTIME_ARR']


merged_df['DepDel'] = merged_df['PLANNEDTIME_DEP'] - merged_df['ACTUALTIME_DEP']


merged_df['Waiting_Time'] = merged_df['ACTUALTIME_DEP'] - merged_df['ACTUALTIME_ARR']

In [ ]:
merged_df

In [ ]:
merged_df.isna().sum()

In [ ]:
#create master cleaned leaving and trips data combined csv
merged_df.to_csv('MasterCleanedLeaving&TripsDataCombined.csv')

In [ ]:
#create master cleaned leaving and trips data combined csv
merged_df.to_feather('MasterCleanedLeaving&TripsDataCombined.feather')

In [ ]:
#change timestamps
actualTime_Arr = []

for row in merged_df['ActualTime_Arr']:
    time = strftime("%H:%M:%S", gmtime(row))
    actualTime_Arr.append(time)
            
actualTime_Arr

In [ ]:
#add new times
merged_df['ActualTime_Arr'] = actualTime_Arr

In [ ]:
actualTime_Dep = []

for row in merged_df['ActualTime_Dep']:
    time = strftime("%H:%M:%S", gmtime(row))
    actualTime_Dep.append(time)
            
actualTime_Dep

In [ ]:
merged_df['ActualTime_Dep'] = actualTime_Dep

In [ ]:
merged_df

In [ ]:
PlannedTime_Arr = []

for row in merged_df['PlannedTime_Arr']:
    time = strftime("%H:%M:%S", gmtime(row))
    PlannedTime_Arr.append(time)
            
PlannedTime_Arr

In [ ]:
merged_df['PlannedTime_Arr'] = PlannedTime_Arr

In [ ]:
PlannedTime_Dep = []

for row in merged_df['PlannedTime_Dep']:
    time = strftime("%H:%M:%S", gmtime(row))
    PlannedTime_Dep.append(time)
            
PlannedTime_Dep

In [ ]:
merged_df['PlannedTime_Dep'] = PlannedTime_Dep

In [ ]:
#add month feature
merged_df['Month'] = merged_df['DayOfService'].dt.month
    

In [ ]:
#add day feature
merged_df['Day'] = merged_df['DayOfService'].dt.dayofweek

In [ ]:
#add schoolfeature

list_School = []

for row in merged_df['Month']:
    if row == 6 or row == 7 or row== 8:
        list_School.append('0')
    else:
        list_School.append('1')       

In [ ]:
merged_df['School']=list_School 

In [ ]:
#add rushhour feature
list_rushHour = []

for row in merged_df['ActualTime_Dep']:
    if row >= '07:00:00' and row <= '08:30:00' or row >= '16:00:00' and row <= '18:00:00':
        list_rushHour.append('1')
    else:
        list_rushHour.append('0')
        

In [ ]:
merged_df['RushHour']=list_rushHour

In [ ]:
#add weekend feature
list_weekend = []

for row in merged_df['Day']:
    if row == '5' or row == '6':
        list_weekend.append('1')
    else:
        list_weekend.append('0')

In [ ]:
merged_df['Weekend'] = list_weekend

In [ ]:
#add holiday feature
holidays = ['2018/01/01', '2018/03/17', '2018/04/02', '2018/05/07','2018/06/04', '2018/08/06', '2018/10/29', '2018/12/25', '2018/12/26']

# for holiday in holidays:
#     list_dfHolidays = []
#     for row in df['DayOfService']:
#         print(row.date())
#         if row.date() == holiday.date():
#             print('true')
# #             list_dfHolidays.append('1')           
#         else:
# #             list_dfHolidays.append('0')
#             print('false')
newHolidays = []
for holiday in holidays:
    newHolidays.append(datetime.datetime.strptime(holiday,"%Y/%m/%d"))


In [ ]:

newhols1 = []

for holiday in newHolidays:
      newhols1.append(holiday.date())

In [ ]:
list_dfHols = []
for row in merged_df['DayOfService']:
#         print(row.date())
    if row.date() == datetime.date(2018, 1, 1) or row.date() == datetime.date(2018, 3, 17) or row.date() == datetime.date(2018, 4, 2) or row.date() == datetime.date(2018, 5, 7) or row.date() == datetime.date(2018, 6, 4) or row.date() == datetime.date(2018, 8, 6) or row.date() == datetime.date(2018, 10, 29) or row.date() ==  datetime.date(2018, 12, 25) or row.date() == datetime.date(2018, 12, 26):
#             print('true')
        list_dfHols.append('1')           
    else:
        list_dfHols.append('0')
#             print('false')

In [ ]:
merged_df['Holiday'] = list_dfHols

In [ ]:
df.tail(500000)